# Metal Abundance Evolution

In [57]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic
import warnings

In [58]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse
from auriga.cosmology import Cosmology

In [59]:
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [60]:
figure_setup()
settings = Settings()

In [61]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [62]:
def read_data(simulation: str, tag_in_situ: bool = False) -> pd.DataFrame:
    cosmology = Cosmology()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_stellar_age()
    s.add_metal_abundance(of="Fe", to='H')
    s.add_metal_abundance(of='O', to='H')
    s.add_metal_abundance(of='O', to="Fe")
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "StellarFormationTime_Gyr": cosmology.expansion_factor_to_time(
            s.stellar_formation_time[is_real_star & is_main_obj]),
        "[Fe/H]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "[O/H]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "[O/Fe]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "xPosition_ckpc": s.pos[is_real_star & is_main_obj, 0],
        "yPosition_ckpc": s.pos[is_real_star & is_main_obj, 1],
        "zPosition_ckpc": s.pos[is_real_star & is_main_obj, 2],
        "ComponentTag": s.region_tag[is_real_star & is_main_obj],
        }

    if tag_in_situ:
        s.tag_in_situ_stars()
        props["IsInSitu"] = s.is_in_situ[is_real_star & is_main_obj]

    return pd.DataFrame(props)

In [63]:
times = np.linspace(0, 14, 50)  # Gyr
bin_width = np.diff(times)[0]  # Gyr
bin_centers = times[1:] - bin_width / 2  # Gyr

In [68]:
simulation = "au6_or_l4_s127"
galaxy = parse(simulation)[0]
label = f"Au{galaxy}"
df = read_data(simulation=simulation)

fig, ax = plt.subplots(figsize=(2.5, 2.5))

ax.tick_params(which='both', direction="in")
ax.set_axisbelow(True)
ax.grid(True, ls='-', lw=0.25, c="gainsboro")

ax.set_xlim(0, 14)
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel("Stellar Formation Time [Gyr]")

ax.set_ylim(-2, 1.5)
ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1.0])
ax.set_ylabel("[Fe/H]")

mean, _, _ = binned_statistic(
    x=df["StellarFormationTime_Gyr"],
    values=df["[Fe/H]"],
    statistic="mean", bins=times)
std, _, _ = binned_statistic(
    x=df["StellarFormationTime_Gyr"],
    values=df["[Fe/H]"],
    statistic=np.nanstd, bins=times)
ax.fill_between(x=bin_centers, y1=mean - std, y2=mean + std,
                color='k', alpha=0.2, lw=0)
ax.plot(bin_centers, mean, c='k', lw=1.0, label="Total", zorder=12)

for i, component in enumerate(settings.components):
    is_component = df["ComponentTag"] == i
    mean, _, _ = binned_statistic(
        x=df["StellarFormationTime_Gyr"][is_component],
        values=df["[Fe/H]"][is_component],
        statistic=np.nanmean, bins=times)
    std, _, _ = binned_statistic(
        x=df["StellarFormationTime_Gyr"][is_component],
        values=df["[Fe/H]"][is_component],
        statistic=np.nanstd, bins=times)
    ax.fill_between(x=bin_centers, y1=mean - std, y2=mean + std,
                    color=settings.component_colors[component],
                    alpha=0.2, lw=0)
    ax.plot(bin_centers, mean, c=settings.component_colors[component],
            lw=1.0, zorder=12, label=settings.component_labels[component])

ax.legend(loc="lower right", framealpha=0.0, fontsize=5.0,
            bbox_to_anchor=(0.99, 0.01), ncol=1)

ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
        size=8.0, transform=ax.transAxes, ha='left', va='top')

fig.savefig(
    f"../images/metallicity_evolution/{simulation}{SUFFIX}.pdf")
plt.close(fig)

In [67]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_ylim(-2, 1.5)
    ax.set_yticks([-1.5, -1, -0.5, 0, 0.5, 1.0])
    ax.set_axisbelow(True)
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel(r"$\tau_\star$ [Gyr]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel("[Fe/H]")

for i, galaxy in enumerate(settings.groups["Included"]):
    ax = axs.flatten()[i]
    simulation = f"au{galaxy}_or_l4_s127"
    label = f"Au{galaxy}"
    df = read_data(simulation=simulation)

    mean, _, _ = binned_statistic(
        x=df["StellarFormationTime_Gyr"],
        values=df["[Fe/H]"],
        statistic="mean", bins=times)
    std, _, _ = binned_statistic(
        x=df["StellarFormationTime_Gyr"],
        values=df["[Fe/H]"],
        statistic=np.nanstd, bins=times)
    ax.fill_between(x=bin_centers, y1=mean - std, y2=mean + std,
                    color='k', alpha=0.2, lw=0)
    ax.plot(bin_centers, mean, c='k', lw=1.0,
            label="Total", zorder=13, ls="--")

    for i, component in enumerate(settings.components):
        is_component = df["ComponentTag"] == i
        mean, _, _ = binned_statistic(
            x=df["StellarFormationTime_Gyr"][is_component],
            values=df["[Fe/H]"][is_component],
            statistic=np.nanmean, bins=times)
        std, _, _ = binned_statistic(
            x=df["StellarFormationTime_Gyr"][is_component],
            values=df["[Fe/H]"][is_component],
            statistic=np.nanstd, bins=times)
        ax.fill_between(x=bin_centers, y1=mean - std, y2=mean + std,
                        color=settings.component_colors[component],
                        alpha=0.2, lw=0)
        ax.plot(bin_centers, mean, c=settings.component_colors[component],
                lw=1.0, zorder=12, label=settings.component_labels[component])

    ax.text(x=0.05, y=0.95,
            s=r"$\texttt{" + label + "}$",
            size=6.0, transform=ax.transAxes,
            ha='left', va='top',
            )

    if galaxy == 10:
        ax.legend(loc="lower right", framealpha=0.0, fontsize=4.0,
                  bbox_to_anchor=(0.99, 0.01), ncol=1)

    fig.savefig(
        f"../images/metallicity_evolution/included{SUFFIX}.pdf")

plt.close(fig)